# Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |

# Problema 2

Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de R\$ 2,00 nas duas primeiras semanas e R\$ 2,50 nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de 5.000, 10.000, 30.000 e 60.000. A capacidade máxima de produção da empresa é de 25.000 camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais 10.000 camisetas em cada semana. Nesse caso, o custo de produção sobe para R\$ 2,80. O excesso de produção pode ser estocado a um custo de R\$ 0,20 por unidade por semana. Formule um modelo que minimize os custos.

# Modelagem matemática

A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

## Declaração dos parâmetros

* $T$: total de semanas.
* $c_{t} \in \mathbb{R}_+$: custo de produção de camisetas na semana $t \in T$.  
* $C_{t} \in \mathbb{R}_+$: custo de produção de camisetas na semana $t \in T$ com hora extra.  
* $d_{t} \in \mathbb{N}$: demanda de camisetas na semana $t \in T$.
* Produção de camisetas por semana: $25.000$.
* Custo de estocagem de camisetas por semana: $R\$0,20$.

In [ ]:
semanas = [1, 2, 3, 4]

custos_de_producao = {
  1: 2,
  2: 2,
  3: 2.5,
  4: 2.5,
}

custos_de_producao_extra = {
  1: 2.8,
  2: 2.8,
  3: 0,
  4: 0,
}

demanda = {
  1: 5000,
  2: 10000,
  3: 30000,
  4: 60000,
}

## Variáveis de decisão

* $x_{t} \ge 0$: quantidade de camisetas produzidas na semana $t \in T$ sem hora extra.
* $y_{t} \ge 0$: quantidade de camisetas produzidas na semana $t \in T$ com hora extra.
* $w_{t} \ge 0$: quantidade de camisetas estocadas no fim da semana $t \in T$.  

## Função objetivo

* Minimização dos custos de produção e estocagem de camisetas: $\min \sum_{t \in T} (c_{t}x_{t} + C_{t}y_{t} + 0,2w_{t})$

## Restrições

- Restrição 1: Número máximo de camisetas que podem ser produzidas por semana.
\begin{equation}
x_t \le 25000
\end{equation}

- Restrição 2: Controle de estoque e demanda
\begin{equation}
x_t + w_{t-1} - d_t \ge e_t
\end{equation}


#Resolvendo o Problema

## Instalação da biblioteca PuLP

In [ ]:
!pip install pulp
import pulp

## Declaração do objeto que representa o modelo matemático

In [ ]:
modelo = pulp.LpProblem('producao_camisetas', pulp.LpMinimize)

## Variáveis de decisão

In [ ]:
producao = pulp.LpVariable.dicts('producao', semanas, lowBound=0)
producao_extra = pulp.LpVariable.dicts('producao_extra', semanas, lowBound=0)
estoque = pulp.LpVariable.dicts('estoque', semanas, lowBound=0)

## Função objetivo

In [ ]:
modelo += pulp.lpSum(custos_de_producao[semana] * producao[semana] + custos_de_producao_extra[semana] * producao_extra[semana] + 0.2 * estoque[semana] for semana in semanas)

## Restrições

- Restrição 1: Número máximo de camisetas que podem ser produzidas por semana.

In [ ]:
for semana in semanas:
  modelo += producao[semana] <= 25000

for semana in semanas:
  modelo += producao_extra[semana] <= 10000

# Não há produção extra nas semanas 3 e 4:
modelo += producao_extra[3] == 0
modelo += producao_extra[4] == 0

- Restrição 2: Controle de estoque e demanda

In [ ]:
for semana in semanas:
  if semana > 1:
    modelo += producao[semana] + producao_extra[semana] + estoque[semana-1] - demanda[semana] == estoque[semana]
  else:
    modelo += producao[semana] + producao_extra[semana] - demanda[semana] == estoque[semana]

### Resolvendo o problema

In [ ]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [ ]:
print('status: ', pulp.LpStatus[status])
print('funcao objetivo: ', modelo.objective.value())
for semana in semanas:
  print('\nsemana:', semana)
  print('camisas produzidas:', producao[semana].value())
  print('camisas produzidas em hora extra:', producao_extra[semana].value())
  print('camisas estocadas:', estoque[semana].value())
  print('custo de produção da semana:', producao[semana].value()*custos_de_producao[semana] + producao_extra[semana].value()*custos_de_producao_extra[semana])

status:  Optimal
funcao objetivo:  258000.0

semana: 1
camisas produzidas: 25000.0
camisas produzidas em hora extra: 0.0
camisas estocadas: 20000.0
custo de produção da semana: 50000.0

semana: 2
camisas produzidas: 25000.0
camisas produzidas em hora extra: 5000.0
camisas estocadas: 40000.0
custo de produção da semana: 64000.0

semana: 3
camisas produzidas: 25000.0
camisas produzidas em hora extra: 0.0
camisas estocadas: 35000.0
custo de produção da semana: 62500.0

semana: 4
camisas produzidas: 25000.0
camisas produzidas em hora extra: 0.0
camisas estocadas: 0.0
custo de produção da semana: 62500.0
